In [2]:
import hashlib
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# Database connection details

conn_params = {
    "dbname": "postgres",
    "user": "postgres.vhlxkellegbbusqersif",
    "password": "MannaBtle*fix",
    "host": "aws-0-us-east-1.pooler.supabase.com",
}

# Function to calculate the hash of a file

def file_hash(file_path):
    with open(file_path, 'rb') as file:
        file_content = file.read()
    return hashlib.sha256(file_content).hexdigest()

# Connect to database

conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

# List of our CSV files and their corresponding column name in the EconomicData table
files_and_columns = [
    ("./econHousingData/ATLUnemployRate.csv", "unemployment_rate"),
    ("./econHousingData/ATLPerCapitaPersonIncome.csv", "per_capita_personal_income"),
    ("./econHousingData/ATLPOPSandySpr.csv", "population"),
    ("./econHousingData/MedianListPriceSqFt.csv", "median_list_price_sq_ft"),
]

for file_path, column_name in files_and_columns:
    # Calculate the hash of the current file
    current_file_hash = file_hash(file_path)
    
    # Check if the file's hash exists in the database
    cur.execute("SELECT EXISTS(SELECT 1 FROM file_hashes WHERE hash = %s)", (current_file_hash,))
    if cur.fetchone()[0]:
        print(f"File {file_path} has already been processed. Skipping...")
        continue
    
    # Load the CSV file into a pandas DataFrame if not skipped
    df = pd.read_csv(file_path)     
    
    # Convert column header to lower case if not already
    df.columns = [col.lower() for col in df.columns]
    
    # Convert data format to YYYY-MM-DD if necessary
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
    
    # Prepare data for insertion
    # Assuming 'region_id' is known, replace '1' with the correct value for our region
    data_tuples = [(row['date'], row[column_name], 1) for index, row in df.iterrows()]
    
    # SQL statement for inserting data
    insert_query = f"""
    INSERT INTO economic_data (date, {column_name}, region_id)
    VALUES %s
    ON CONFLICT (date, region_id)
    DO UPDATE SET {column_name} = EXCLUDED.{column_name}
    """
    
    # Execute the insert operation
    execute_values(cur, insert_query, data_tuples)
    
    # Insert the file's hash into the file_hashes table
    cur.execute("INSERT INTO file_hashes (hash) VALUES (%s) ON CONFLICT (hash) DO NOTHING", (current_file_hash,))

    
    # Commit the transaction
    conn.commit()
    
    
cur.close()
conn.close()

File ./econHousingData/ATLUnemployRate.csv has already been processed. Skipping...
File ./econHousingData/ATLPerCapitaPersonIncome.csv has already been processed. Skipping...
File ./econHousingData/ATLPOPSandySpr.csv has already been processed. Skipping...
File ./econHousingData/MedianListPriceSqFt.csv has already been processed. Skipping...
